In [4]:
import pygame, sys
from pygame.locals import *
import random
import numpy as np

class game_env:
    def __init__(self,suffix):
        self.q_table = np.zeros((100,4))
        self.reward_map = {'traffic.png':-20, 'road.png':-3,'jam.png':-50,'fast.png':0,'house.png':500,'man.png':-500,'already_visited':-10,'invalid':-100}
        self.dir = {0:'left',2:'right',1:'down',3:'up'}
        self.alpha = 0.75
        self.beta = 0.75
        self.greedy = 0
        self.random = 1
        self.delta = 0.005
        self.game_dim = (500,650)
        self.text_space = 150
        self.initial_cood = (0,0+self.text_space)
        self.rows,self.columns = 10,10
        self.start_state = 0
        self.end_state = 99
        self.cell_dim = self.game_dim[0]/self.rows
        self.final_cood = (self.game_dim[0]-self.cell_dim, self.game_dim[1]-self.cell_dim)
        self.game_grid = self.new_game_env()
        self.suffix = suffix
        self.action_space = {0:{'x':-1*self.cell_dim,'y':0}, 2:{'x':self.cell_dim,'y':0}, 1:{'x':0,'y':self.cell_dim},3:{'x':0,'y':-1*self.cell_dim}}
        try:
            with open('env_weights\\weights_{}.npy'.format(self.suffix),'rb') as f:
                                self.q_table = np.load(f)
            with open('env_weights\\env_{}.npy'.format(self.suffix),'rb') as f:
                                self.game_grid = np.load(f)
        except Exception as e:
            print('No such files pre-exists. Starting a new environment')
            with open('env_weights\\env_{}.npy'.format(self.suffix),'wb') as f:
                                np.save(f,self.game_grid)
            with open('env_weights\\weights_{}.npy'.format(self.suffix),'wb') as f:
                                np.save(f,self.q_table)
            pass

    
    def new_game_env(self):
        matrix = random.choices(['road.png','traffic.png','jam.png','fast.png'],weights=[0.55,0.15,0.15,0.15], k=self.rows*self.columns)
        matrix = np.asarray(matrix).reshape(self.rows,self.columns)
        matrix[0][0] = 'man.png'
        matrix[self.rows-1][self.columns-1] = 'house.png'
        return matrix
    
    def image_loader(self,img_path):
        img = pygame.image.load('icons\\{}'.format(img_path))
        img = pygame.transform.scale(img,(self.cell_dim,self.cell_dim))
        return img
    
    def print_summary(self,text,cood,size):
        font = pygame.font.Font(pygame.font.get_default_font(), size)
        text_surface = font.render(text, True, (255,255,255))
        DISPLAYSURF.blit(text_surface,cood)
    
    def initial_state(self):
        DISPLAYSURF.fill((0,0,0))
        for x in range(self.rows):
                for y in range(self.columns):
                    img = self.image_loader(self.game_grid[x][y])
                    cood = (y*self.cell_dim,x*self.cell_dim+self.text_space)
                    DISPLAYSURF.blit(img,cood)
        self.print_summary('Traffic Turbo',(175,25),24)
        pygame.display.update()

    def steps_visualizer(self,cood):
        img = pygame.image.load('icons\\feet.png')
        img = pygame.transform.scale(img,(self.cell_dim,self.cell_dim))
        print(cood)
        DISPLAYSURF.blit(img,cood)
        pygame.display.update()
        clock.tick(1)    
    
    def cood_state_calc(self,cood):
        
        state = int((self.rows*(cood[1]-self.text_space)/self.cell_dim)+(cood[0]/self.cell_dim)) 
        return state
    
    def state_cood_calc(self, state):
        cood = int((state%self.rows)*self.cell_dim),int((state//self.rows)*self.cell_dim+self.text_space)
        return cood
    
    def is_valid_move(self, cood, already_visited):
        
        if cood in already_visited:
            return False
        
        if self.initial_cood[0]<=cood[0]<=self.final_cood[0] and self.initial_cood[1]<=cood[1]<=self.final_cood[1]:
                return True
        return False
        
    
    def q_table_update(self,  state, action, already_visited):
        
        curr_cood = self.state_cood_calc(state)
        new_cood = (int(curr_cood[0] + self.action_space[action]['x']), int(curr_cood[1] + self.action_space[action]['y']))
        new_state = self.cood_state_calc(new_cood)
        is_valid = self.is_valid_move(new_cood, already_visited)
                                     
      
        if is_valid:
            reward = self.reward_map[self.game_grid[int(new_state//self.rows)][int(new_state%self.rows)]]
        elif new_cood in already_visited:
            reward = self.reward_map['already_visited']
        else:
            reward = self.reward_map['invalid']
        
        if is_valid:
            print({'action':action,'state':state,'new_cood':new_cood,'curr_cood':curr_cood,'new_state':new_state,'reward':reward,'feature':self.game_grid[int(new_state//10)][int(new_state%10)]})
        
        try:
            state_value_diff = max(self.q_table[new_state]) - self.q_table[state][action]
        except:
            state_value_diff = 0
        self.q_table[state][action]+=self.alpha*(reward + self.beta*state_value_diff)
                                     
        return is_valid, new_state, new_cood,reward
    
    def episode(self, current_state, is_valid):
        pygame.event.get()
        cood = self.state_cood_calc(current_state)
        already_visited = [cood]
        self.steps_visualizer(cood)
        
        while current_state!=self.end_state and is_valid==True:
            pygame.draw.rect(DISPLAYSURF,(0,0,0),(0,100,self.game_dim[0],50))
            pygame.display.update()
            for event in pygame.event.get():
                if event.type==QUIT:
                                pygame.quit()
                                raise Exception('training ended')
            choice = random.choices([True,False],weights=[self.greedy,self.random],k=1)
            if choice[0]:
                     action = np.argmax(self.q_table[current_state])
            else:
                     action = random.choices([0,1,2,3],weights=[0.25,0.25,0.25,0.25],k=1)
                     action = action[0]
            self.print_summary('State:{}'.format(current_state),(10,100),15)
            self.print_summary('Action:{}'.format(self.dir[action]),(110,100),15)
            is_valid, current_state, cood, reward = self.q_table_update(current_state, action, already_visited)
            
            self.print_summary('Reward:{}'.format(reward),(220,100),15)
            
            if is_valid==False and cood not in already_visited:
                self.print_summary('INVALID MOVE !!',(330,100),15)
            elif is_valid==False:
                self.print_summary('ALREADY VISITED',(330,100),15)
            else:
                self.print_summary('New State:{}'.format(current_state),(330,100),15)
            
            pygame.display.update()
            clock.tick(0.9)
            already_visited.append(cood)
            if is_valid:
                self.steps_visualizer(cood)
            else:
                break
            
    
    def training(self, epoch):
                    state=random.randint(self.start_state,self.end_state)
                    self.initial_state()
                    self.print_summary(' Episode:{}'.format(epoch),(200,60),20)
                    self.episode(state, True)  
                    print('episode {} ---->'.format(epoch))
                    pygame.display.set_caption('greedy={}, random={}'.format(round(self.greedy,4),round(self.random,4)))
                    if epoch%50==0:
                        if self.random>0:
                                self.greedy+=self.delta
                                self.random-=self.delta
                                self.greedy = min(self.greedy,1)
                                self.random= max(self.random,0)
                        print({'greedy':self.greedy,'random':self.random})
                    if epoch%2000==0:
                        self.delta*=2
                        with open('env_weights\\weights_{}.npy'.format(self.suffix),'wb') as f:
                            np.save(f,self.q_table)
                    
                    clock.tick(1)
    
    def testing(self,initial_state=0):
            self.greedy = 1
            self.random = 0
            with open('env_weights\\env_{}.npy'.format(self.suffix),'rb') as f:
                self.game_grid = np.load(f)
            
            with open('env_weights\\weights_{}.npy'.format(self.suffix),'rb') as f:
                self.q_table = np.load(f)
            
            self.initial_state()
            self.episode(initial_state,True)
        

In [5]:
game = game_env('final_v')

In [6]:
pygame.init()
count=1
flag = 0

DISPLAYSURF = pygame.display.set_mode(game.game_dim,0,32)
clock = pygame.time.Clock()

while True:
    game.training(count)
    count+=1
    if count%5000==0:
        pygame.quit()
        break
    for event in pygame.event.get():
                if event.type==QUIT:
                            pygame.quit()
                            flag = 1
                            raise Exception('game ended')
    if flag==1:
        break
    

(100, 350)
{'action': 0, 'state': 42, 'new_cood': (50, 350), 'curr_cood': (100, 350), 'new_state': 41, 'reward': -20, 'feature': 'traffic.png'}
(50, 350)
{'action': 0, 'state': 41, 'new_cood': (0, 350), 'curr_cood': (50, 350), 'new_state': 40, 'reward': -3, 'feature': 'road.png'}
(0, 350)
{'action': 1, 'state': 40, 'new_cood': (0, 400), 'curr_cood': (0, 350), 'new_state': 50, 'reward': 0, 'feature': 'fast.png'}
(0, 400)
episode 1 ---->
(250, 350)
{'action': 2, 'state': 45, 'new_cood': (300, 350), 'curr_cood': (250, 350), 'new_state': 46, 'reward': -3, 'feature': 'road.png'}
(300, 350)
{'action': 1, 'state': 46, 'new_cood': (300, 400), 'curr_cood': (300, 350), 'new_state': 56, 'reward': 0, 'feature': 'fast.png'}
(300, 400)
episode 2 ---->
(350, 150)
{'action': 2, 'state': 7, 'new_cood': (400, 150), 'curr_cood': (350, 150), 'new_state': 8, 'reward': -3, 'feature': 'road.png'}
(400, 150)
{'action': 1, 'state': 8, 'new_cood': (400, 200), 'curr_cood': (400, 150), 'new_state': 18, 'reward': 

(50, 400)
{'action': 1, 'state': 51, 'new_cood': (50, 450), 'curr_cood': (50, 400), 'new_state': 61, 'reward': -3, 'feature': 'road.png'}
(50, 450)
episode 20 ---->
(0, 450)
episode 21 ---->
(400, 450)
{'action': 2, 'state': 68, 'new_cood': (450, 450), 'curr_cood': (400, 450), 'new_state': 69, 'reward': -3, 'feature': 'road.png'}
(450, 450)
{'action': 1, 'state': 69, 'new_cood': (450, 500), 'curr_cood': (450, 450), 'new_state': 79, 'reward': -3, 'feature': 'road.png'}
(450, 500)
episode 22 ---->
(50, 600)
{'action': 2, 'state': 91, 'new_cood': (100, 600), 'curr_cood': (50, 600), 'new_state': 92, 'reward': -50, 'feature': 'jam.png'}
(100, 600)
episode 23 ---->
(0, 300)
episode 24 ---->
(300, 400)
{'action': 3, 'state': 56, 'new_cood': (300, 350), 'curr_cood': (300, 400), 'new_state': 46, 'reward': -3, 'feature': 'road.png'}
(300, 350)
{'action': 0, 'state': 46, 'new_cood': (250, 350), 'curr_cood': (300, 350), 'new_state': 45, 'reward': -3, 'feature': 'road.png'}
(250, 350)
{'action': 1,

(400, 450)
{'action': 0, 'state': 68, 'new_cood': (350, 450), 'curr_cood': (400, 450), 'new_state': 67, 'reward': -3, 'feature': 'road.png'}
(350, 450)
episode 36 ---->
(150, 200)
{'action': 3, 'state': 13, 'new_cood': (150, 150), 'curr_cood': (150, 200), 'new_state': 3, 'reward': -3, 'feature': 'road.png'}
(150, 150)
episode 37 ---->
(250, 400)
{'action': 0, 'state': 55, 'new_cood': (200, 400), 'curr_cood': (250, 400), 'new_state': 54, 'reward': 0, 'feature': 'fast.png'}
(200, 400)
{'action': 1, 'state': 54, 'new_cood': (200, 450), 'curr_cood': (200, 400), 'new_state': 64, 'reward': -3, 'feature': 'road.png'}
(200, 450)
episode 38 ---->
(400, 400)
{'action': 2, 'state': 58, 'new_cood': (450, 400), 'curr_cood': (400, 400), 'new_state': 59, 'reward': -3, 'feature': 'road.png'}
(450, 400)
episode 39 ---->
(350, 300)
{'action': 3, 'state': 37, 'new_cood': (350, 250), 'curr_cood': (350, 300), 'new_state': 27, 'reward': -3, 'feature': 'road.png'}
(350, 250)
{'action': 2, 'state': 27, 'new_c

(200, 300)
{'action': 0, 'state': 34, 'new_cood': (150, 300), 'curr_cood': (200, 300), 'new_state': 33, 'reward': -3, 'feature': 'road.png'}
(150, 300)
{'action': 3, 'state': 33, 'new_cood': (150, 250), 'curr_cood': (150, 300), 'new_state': 23, 'reward': -3, 'feature': 'road.png'}
(150, 250)
{'action': 2, 'state': 23, 'new_cood': (200, 250), 'curr_cood': (150, 250), 'new_state': 24, 'reward': -20, 'feature': 'traffic.png'}
(200, 250)
{'action': 2, 'state': 24, 'new_cood': (250, 250), 'curr_cood': (200, 250), 'new_state': 25, 'reward': -20, 'feature': 'traffic.png'}
(250, 250)
episode 58 ---->
(200, 550)
{'action': 2, 'state': 84, 'new_cood': (250, 550), 'curr_cood': (200, 550), 'new_state': 85, 'reward': -3, 'feature': 'road.png'}
(250, 550)
{'action': 2, 'state': 85, 'new_cood': (300, 550), 'curr_cood': (250, 550), 'new_state': 86, 'reward': -3, 'feature': 'road.png'}
(300, 550)
{'action': 1, 'state': 86, 'new_cood': (300, 600), 'curr_cood': (300, 550), 'new_state': 96, 'reward': -50,

(100, 450)
{'action': 3, 'state': 62, 'new_cood': (100, 400), 'curr_cood': (100, 450), 'new_state': 52, 'reward': -3, 'feature': 'road.png'}
(100, 400)
{'action': 3, 'state': 52, 'new_cood': (100, 350), 'curr_cood': (100, 400), 'new_state': 42, 'reward': -3, 'feature': 'road.png'}
(100, 350)
{'action': 0, 'state': 42, 'new_cood': (50, 350), 'curr_cood': (100, 350), 'new_state': 41, 'reward': -20, 'feature': 'traffic.png'}
(50, 350)
episode 74 ---->
(100, 300)
{'action': 1, 'state': 32, 'new_cood': (100, 350), 'curr_cood': (100, 300), 'new_state': 42, 'reward': -3, 'feature': 'road.png'}
(100, 350)
{'action': 0, 'state': 42, 'new_cood': (50, 350), 'curr_cood': (100, 350), 'new_state': 41, 'reward': -20, 'feature': 'traffic.png'}
(50, 350)
{'action': 3, 'state': 41, 'new_cood': (50, 300), 'curr_cood': (50, 350), 'new_state': 31, 'reward': -3, 'feature': 'road.png'}
(50, 300)
episode 75 ---->
(0, 150)
episode 76 ---->
(50, 600)
{'action': 3, 'state': 91, 'new_cood': (50, 550), 'curr_cood'

...

{'action': 3, 'state': 15, 'new_cood': (250, 150), 'curr_cood': (250, 200), 'new_state': 5, 'reward': -50, 'feature': 'jam.png'}
(250, 150)
episode 2419 ---->
(150, 250)
{'action': 3, 'state': 23, 'new_cood': (150, 200), 'curr_cood': (150, 250), 'new_state': 13, 'reward': 0, 'feature': 'fast.png'}
(150, 200)
{'action': 2, 'state': 13, 'new_cood': (200, 200), 'curr_cood': (150, 200), 'new_state': 14, 'reward': -3, 'feature': 'road.png'}
(200, 200)
episode 2420 ---->
(50, 600)
{'action': 2, 'state': 91, 'new_cood': (100, 600), 'curr_cood': (50, 600), 'new_state': 92, 'reward': -50, 'feature': 'jam.png'}
(100, 600)
{'action': 3, 'state': 92, 'new_cood': (100, 550), 'curr_cood': (100, 600), 'new_state': 82, 'reward': -20, 'feature': 'traffic.png'}
(100, 550)
{'action': 3, 'state': 82, 'new_cood': (100, 500), 'curr_cood': (100, 550), 'new_state': 72, 'reward': 0, 'feature': 'fast.png'}
(100, 500)
{'action': 0, 'state': 72, 'new_cood': (50, 500), 'curr_cood': (100, 500), 'new_state': 71, 're

(300, 350)
{'action': 2, 'state': 46, 'new_cood': (350, 350), 'curr_cood': (300, 350), 'new_state': 47, 'reward': -3, 'feature': 'road.png'}
(350, 350)
{'action': 1, 'state': 47, 'new_cood': (350, 400), 'curr_cood': (350, 350), 'new_state': 57, 'reward': -3, 'feature': 'road.png'}
(350, 400)
episode 2439 ---->
(400, 250)
{'action': 0, 'state': 28, 'new_cood': (350, 250), 'curr_cood': (400, 250), 'new_state': 27, 'reward': -3, 'feature': 'road.png'}
(350, 250)
{'action': 1, 'state': 27, 'new_cood': (350, 300), 'curr_cood': (350, 250), 'new_state': 37, 'reward': -3, 'feature': 'road.png'}
(350, 300)
episode 2440 ---->
(150, 400)
{'action': 0, 'state': 53, 'new_cood': (100, 400), 'curr_cood': (150, 400), 'new_state': 52, 'reward': -3, 'feature': 'road.png'}
(100, 400)
episode 2441 ---->
(200, 400)
{'action': 1, 'state': 54, 'new_cood': (200, 450), 'curr_cood': (200, 400), 'new_state': 64, 'reward': -3, 'feature': 'road.png'}
(200, 450)
{'action': 2, 'state': 64, 'new_cood': (250, 450), 'c

(350, 400)
{'action': 0, 'state': 57, 'new_cood': (300, 400), 'curr_cood': (350, 400), 'new_state': 56, 'reward': 0, 'feature': 'fast.png'}
(300, 400)
{'action': 1, 'state': 56, 'new_cood': (300, 450), 'curr_cood': (300, 400), 'new_state': 66, 'reward': -3, 'feature': 'road.png'}
(300, 450)
episode 2454 ---->
(300, 250)
{'action': 1, 'state': 26, 'new_cood': (300, 300), 'curr_cood': (300, 250), 'new_state': 36, 'reward': -3, 'feature': 'road.png'}
(300, 300)
{'action': 0, 'state': 36, 'new_cood': (250, 300), 'curr_cood': (300, 300), 'new_state': 35, 'reward': -50, 'feature': 'jam.png'}
(250, 300)
{'action': 3, 'state': 35, 'new_cood': (250, 250), 'curr_cood': (250, 300), 'new_state': 25, 'reward': -20, 'feature': 'traffic.png'}
(250, 250)
episode 2455 ---->
(250, 450)
{'action': 0, 'state': 65, 'new_cood': (200, 450), 'curr_cood': (250, 450), 'new_state': 64, 'reward': -3, 'feature': 'road.png'}
(200, 450)
episode 2456 ---->
(450, 500)
{'action': 3, 'state': 79, 'new_cood': (450, 450),

(350, 200)
{'action': 1, 'state': 17, 'new_cood': (350, 250), 'curr_cood': (350, 200), 'new_state': 27, 'reward': -3, 'feature': 'road.png'}
(350, 250)
{'action': 0, 'state': 27, 'new_cood': (300, 250), 'curr_cood': (350, 250), 'new_state': 26, 'reward': -20, 'feature': 'traffic.png'}
(300, 250)
episode 2465 ---->
(100, 200)
{'action': 1, 'state': 12, 'new_cood': (100, 250), 'curr_cood': (100, 200), 'new_state': 22, 'reward': -50, 'feature': 'jam.png'}
(100, 250)
{'action': 1, 'state': 22, 'new_cood': (100, 300), 'curr_cood': (100, 250), 'new_state': 32, 'reward': -3, 'feature': 'road.png'}
(100, 300)
{'action': 1, 'state': 32, 'new_cood': (100, 350), 'curr_cood': (100, 300), 'new_state': 42, 'reward': -3, 'feature': 'road.png'}
(100, 350)
{'action': 2, 'state': 42, 'new_cood': (150, 350), 'curr_cood': (100, 350), 'new_state': 43, 'reward': -50, 'feature': 'jam.png'}
(150, 350)
{'action': 2, 'state': 43, 'new_cood': (200, 350), 'curr_cood': (150, 350), 'new_state': 44, 'reward': -3, 'f

Exception: training ended

In [7]:
game = game_env('final_v')

In [9]:
flag = 0

pygame.init()
DISPLAYSURF = pygame.display.set_mode(game.game_dim,0,32)
pygame.display.set_caption('Lucky The Racer')
clock = pygame.time.Clock()
while True:
    game.testing()
    for event in pygame.event.get():
                if event.type==QUIT:
                            pygame.quit()
                            flag = 1
                            raise Exception('game ended')
    if flag==1:
        break

(0, 150)
{'action': 1, 'state': 0, 'new_cood': (0, 200), 'curr_cood': (0, 150), 'new_state': 10, 'reward': -3, 'feature': 'road.png'}
(0, 200)
{'action': 1, 'state': 10, 'new_cood': (0, 250), 'curr_cood': (0, 200), 'new_state': 20, 'reward': -3, 'feature': 'road.png'}
(0, 250)
{'action': 1, 'state': 20, 'new_cood': (0, 300), 'curr_cood': (0, 250), 'new_state': 30, 'reward': 0, 'feature': 'fast.png'}
(0, 300)
{'action': 1, 'state': 30, 'new_cood': (0, 350), 'curr_cood': (0, 300), 'new_state': 40, 'reward': -3, 'feature': 'road.png'}
(0, 350)
{'action': 1, 'state': 40, 'new_cood': (0, 400), 'curr_cood': (0, 350), 'new_state': 50, 'reward': 0, 'feature': 'fast.png'}
(0, 400)
{'action': 2, 'state': 50, 'new_cood': (50, 400), 'curr_cood': (0, 400), 'new_state': 51, 'reward': -3, 'feature': 'road.png'}
(50, 400)
{'action': 2, 'state': 51, 'new_cood': (100, 400), 'curr_cood': (50, 400), 'new_state': 52, 'reward': -3, 'feature': 'road.png'}
(100, 400)
{'action': 2, 'state': 52, 'new_cood': (15

(450, 550)
{'action': 1, 'state': 89, 'new_cood': (450, 600), 'curr_cood': (450, 550), 'new_state': 99, 'reward': 500, 'feature': 'house.png'}
(450, 600)
(0, 150)
{'action': 1, 'state': 0, 'new_cood': (0, 200), 'curr_cood': (0, 150), 'new_state': 10, 'reward': -3, 'feature': 'road.png'}
(0, 200)
{'action': 1, 'state': 10, 'new_cood': (0, 250), 'curr_cood': (0, 200), 'new_state': 20, 'reward': -3, 'feature': 'road.png'}
(0, 250)
{'action': 1, 'state': 20, 'new_cood': (0, 300), 'curr_cood': (0, 250), 'new_state': 30, 'reward': 0, 'feature': 'fast.png'}
(0, 300)
{'action': 1, 'state': 30, 'new_cood': (0, 350), 'curr_cood': (0, 300), 'new_state': 40, 'reward': -3, 'feature': 'road.png'}
(0, 350)
{'action': 1, 'state': 40, 'new_cood': (0, 400), 'curr_cood': (0, 350), 'new_state': 50, 'reward': 0, 'feature': 'fast.png'}
(0, 400)
{'action': 2, 'state': 50, 'new_cood': (50, 400), 'curr_cood': (0, 400), 'new_state': 51, 'reward': -3, 'feature': 'road.png'}
(50, 400)


Exception: training ended